# Décodage du charabia avec linguistica
Produire trois version du texte avec :
- la phrase d'origine
- la phrase découpée
- la glose pour chaque morphème

À terme produire un document LaTeX avec en plus une ligne pour noter la traduction

In [1]:
import linguistica as lxa
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import re,tabulate
from itertools import combinations
lxa.__version__  # show version number

'5.2.1'

## Lire le corpus

In [2]:
charabia="Aveugle"
repName="/Users/gilles/ownCloud/Cours/Bordeaux/M2-TraitementsCorpus/"
fName="Corpus%s-Sentenced-Ordered.txt"%charabia
# lxaCorpus = lxa.read_corpus(repName+fName)

In [3]:
texteBrut=pd.read_csv(repName+fName,encoding="utf8",header=None,sep="\t").values.tolist()
texteBrut=[t[0] for t in texteBrut]
compW=re.compile(u"(\W)+",re.U)
texteBrut=[re.sub(compW," ",l) for l in texteBrut]
lxaCorpus = lxa.from_corpus(texteBrut)

In [31]:
%store -r conteLines

lineConte={}
nContes=[]
nConte=""
nDebutConte=0
for nLine,line in enumerate(texteBrut):
    m=re.search(u"^(\d+)",line)
    if m:
        nConte=m.group(1)
        nContes.append(nConte)
    lineConte[nLine]=nConte
lineConte

{0: u'001',
 1: u'001',
 2: u'001',
 3: u'001',
 4: u'001',
 5: u'001',
 6: u'001',
 7: u'001',
 8: u'001',
 9: u'001',
 10: u'001',
 11: u'001',
 12: u'001',
 13: u'001',
 14: u'001',
 15: u'001',
 16: u'001',
 17: u'001',
 18: u'001',
 19: u'001',
 20: u'001',
 21: u'001',
 22: u'001',
 23: u'001',
 24: u'001',
 25: u'001',
 26: u'001',
 27: u'001',
 28: u'001',
 29: u'001',
 30: u'001',
 31: u'001',
 32: u'001',
 33: u'001',
 34: u'001',
 35: u'001',
 36: u'001',
 37: u'001',
 38: u'001',
 39: u'001',
 40: u'001',
 41: u'001',
 42: u'001',
 43: u'001',
 44: u'001',
 45: u'001',
 46: u'001',
 47: u'001',
 48: u'001',
 49: u'001',
 50: u'001',
 51: u'001',
 52: u'001',
 53: u'001',
 54: u'001',
 55: u'001',
 56: u'001',
 57: u'001',
 58: u'001',
 59: u'001',
 60: u'001',
 61: u'001',
 62: u'001',
 63: u'001',
 64: u'001',
 65: u'001',
 66: u'001',
 67: u'001',
 68: u'001',
 69: u'001',
 70: u'001',
 71: u'001',
 72: u'001',
 73: u'001',
 74: u'001',
 75: u'001',
 76: u'001',
 77: u'00

In [5]:
 tradContes=set(nContes).intersection(set(conteLines))

## Basic information

In [6]:
print "mots-occurrences",lxaCorpus.number_of_word_tokens()
print "mots-types",lxaCorpus.number_of_word_types()

mots-occurrences 206136
mots-types 8720


## Séquences fréquentes
Liste des séquences de mots au dessus du *seuil* de fréquence pour :
- les mots seuls
- les bigrammes
- les trigrammes

In [7]:
def spreadListe(liste,cols):
    result=[]
    if isinstance(liste[0][0],tuple):
        locListe=[" ".join(seq)+" : "+unicode(nb) for seq,nb in liste]
    else:
        locListe=[seq+" : "+unicode(nb) for seq,nb in liste]
#     print locListe
    pos=[None for c in range(cols)]
    for n,l in enumerate(locListe):
        i=n%cols
        pos[i]=l
        if i+1==cols:
            result.append(pos)
            pos=[None for c in range(cols)]
    print (tabulate.tabulate(result))
    return 

In [8]:
seuilUnigram=1000
dUnigram=lxaCorpus.word_unigram_counter()
pUnigram=sorted(dUnigram.items(), key=lambda kv: (kv[1], kv[0]), reverse=True)
print "mots de fréquence supérieure à %d"%seuilUnigram
freqUnigrams=[]
for w,f in pUnigram:
    if f>seuilUnigram:
#         print w,f
        freqUnigrams.append((w,f))
    else:
        break
spreadListe(freqUnigrams,6)

mots de fréquence supérieure à 1000
----------  ------------  ----------  ------------  -----------  ----------
ed : 10267  ñax : 5018    o : 4114    zix : 4051    zih : 4028   ah : 3784
ix : 3553   hoy : 3434    wta : 3101  tid : 2670    ossi : 2623  raq : 2604
zi : 2588   ñux : 2575    vta : 2554  wadzi : 2396  ziq : 2331   za : 2210
ap : 2035   cix : 1941    zu : 1935   ciz : 1836    wod : 1763   iq : 1738
peq : 1590  ñarzi : 1466  ñac : 1439  sec : 1392    xu : 1374    ih : 1320
----------  ------------  ----------  ------------  -----------  ----------


In [9]:
seuilBigram=200
dBigram=lxaCorpus.word_bigram_counter()
pBigram=sorted(dBigram.items(), key=lambda kv: (kv[1], kv[0]), reverse=True)
print "séquences de deux mots de fréquence supérieure à %d"%seuilBigram
freqBigram=[]
for w,f in pBigram:
    if f>seuilBigram:
#         print " ".join(w),f
        freqBigram.append((w,f))
    else:
        break
spreadListe(freqBigram,4)

séquences de deux mots de fréquence supérieure à 200
---------------  --------------  -------------  ---------------
hoy ah : 525     ed wadzi : 516  za ñax : 494   ed za : 469
peq ah : 454     zih iq : 430    zih raq : 399  wadzi ñax : 398
ziq raq : 392    ah ñax : 368    oxz o : 358    zix dacyi : 299
ñax ñarzi : 291  o ix : 258      za zi : 252    ed wta : 250
tid ñaq : 249    ap whiz : 243   za ñux : 233   ed dos : 227
ed zi : 223      ñix hoy : 222   ev ed : 214    hoy ix : 212
---------------  --------------  -------------  ---------------


In [10]:
seuilTrigram=50
dTrigram=lxaCorpus.word_trigram_counter()
pTrigram=sorted(dTrigram.items(), key=lambda kv: (kv[1], kv[0]), reverse=True)
print "séquences de trois mots de fréquence supérieure à %d"%seuilTrigram
freqTrigram=[]
for w,f in pTrigram:
    if f>seuilTrigram:
#         print " ".join(w),f
        freqTrigram.append((w,f))
    else:
        break
spreadListe(freqTrigram,4)

séquences de trois mots de fréquence supérieure à 50
-----------------  --------------------  -----------------  --------------------
ed za ñax : 151    ziq raq ixdaxd : 118  za ñax sec : 109   raq ixdaxd ix : 107
wod hoy ah : 94    riz wozix ap : 85     ed za ñux : 84     ah zih raq : 79
uz o wserix : 71   za ñax ñarzi : 68     yer hoy ah : 66    o ix paqh : 65
ed za zi : 62      za zi ñarzi : 59      o wserix ed : 58   ah ñax wsuyyi : 58
ed wahhi wod : 56  ed wadzi hoy : 56     za ñax wta : 55    wadzi ñax hoy : 55
za zi sec : 54     za ñax raq : 53       ykwh hoy ah : 53   wadzi zix dacyi : 53
ed hoy wozwh : 53  cix za ñax : 53       tid wsay xes : 51  ed wadzi ah : 51
-----------------  --------------------  -----------------  --------------------


# Découpage

In [11]:
dictBrokenWords=lxaCorpus.broken_words_left_to_right()
dictGlossWords={k:"-".join(v) for k,v in dictBrokenWords.iteritems()}
dictHyphenWords={k:"-".join(v) for k,v in dictBrokenWords.iteritems()}

In [294]:
dictFoundGloss={
    "zesheq":"docteur[nom1]",
    "zeshiq":"docteur[nom2]",
    "zesheqix":"docteur-DEF",
    "zad":u"jour",
    "teti":u"œil",
    "ixteti":u"1-œil",
    "heteti":u"2-œil",
    "hqiteti":u"3-œil",
    u"ñocyix":"ciel-DEF",
    "taxzix":"esprit-DEF",
    "wyaxdi":"serpent",
    "pyawsix":"bouteille-DEF",
    "ayrozixzi":"universel[adj]",
    "jexzix":"paysan-DEF",
    "jexzi":"paysan",
    "vixdi":"seigneur?",
    "vixdixi":"seigneur?",
    "wtex":"fils",
    "wtexxiq":"fils-PL",
    "ytexxi-h":"fils?",
    "zahhiq":"fille",
    "vodix":"jeune.fille-DEF",
    "jqtezqi":u"frère",
    "jqeqyoy":u"frérot",
    "wtewhiqyoy":u"sœurette",
    "zqixdi":u"garçon",
    "paq":u"père",
    "jaq":u"enfant",
    "caxz":"homme",
    "caxzix":"homme-DEF",
    "caxzixw":"homme-DEF-POSS",
    "sexi":"femme",
    "sexix":"femme-DEF",
    "pudy":"oiseau",
    "pudyix":"oiseau-DEF",
    "udyix":"chouette-DEF",
    "qarxi":"corbeau-PL?",
    "pqte":"grenouille",
    "pqteix":"grenouille-DEF",
    "dtiwwixi":"oie-DEF-PL",
    "sah":"chat",
    "sahhix":"chat-DEF",
    "cuw":"souris",    
    "cuwix":"souris-DEF",
    "diz":u"chèvre",
    "dizix":u"chèvre-DEF",
    "dizisoz":u"chevreau-PL",
    "uyrix":u"loup-DEF",
    "jtteqxix":u"ours-DEF",
    "tiwiy":u"âne",
    "qtirix":u"renard-DEF",
    "yuwix":u"pou-DEF",
    "yevvix":u"puce-DEF",
    "wser":u"forêt",
    "wserix":u"forêt-DEF",
    "sudyi":u"balle",
    "sudyix":u"balle-DEF",
    "sqexi":u"couronne",
    "wuyhax":"sultan",
    "sexdi":"roi",
    "sexdix":"roi-DEF",
    "zttiriyix":"diable-DEF",
    "zttiriyixw":"diable-DEF-GEN",
    "duyz":"or",
    "cizowhiqvteywi":"saucisse",
    "cizowhiqvteywix":"saucisse",
    "zahhiqix":u"château-DEF",
    "qtezñtihhi":"rouge+chaperon",
    "wyaxdijyazi":"serpent+feuille",
    "pqtesexdix":"grenouille+roi-DEF",
    "sexdizahhiqix":u"roi+château-DEF",
    "sexdiwtexxixw":"roi+fils?",
    "wtexxiwtexxix":"fils+fils?",
    "dyawsowhix":u"verre+cercueil-DEF",
    "duyzsudyi":"or+balle",
    "duyzsqexi":"or+couronne",
    "piqzoxaxz":"Fernand",
    "awsiveh":u"Cendrillon",
    "iywi":u"Élise",
    "qavuxgiy":u"Raiponce",
    "teqoxzi":u"Jorinde",
    "teqoxdiy":u"Joringel",
    u"wxiñrozi":u"Blanche-Neige",
    "sowhiq":"cercueil",
    "ctaxix":"lune",
    "wvoyyicaxz":"violon",
    "wixdi":"lit",
    "jtidiq":"gobelet",
    "yadixiq":"drap-PL",
    "cuxz":"bouche",
    "cuxz":"bouche-DEF",
    "wvqed":"langue",
    "wsqtizziq":"tailleur",
    "wsqtizziqix":"tailleur-DEF",
    "powsiqix":u"pêcheur-DEF",
    "ciwhiq":"patron",
    "soqsidtaqzix":u"cimetière-DEF",
    "soqsi":u"église",
    "soqsix":u"église",
    "wteccih":"clou-DEF",
    "ptiq":"plume-PL",
    "jtexxix":u"haricot-DEF",
    "dxowhix":u"braise-DEF",
    "daxd":"temps",
    "ctiys":"lait",
    "xowwiqxi":"lutin-PL",
    "raxz":"eau",
    "zixdaxd":"temps[DEF.PL]",
    "ixdaxd":"temps[INDEF?]",
    u"ñtixziq":"main",
    u"ñrozi":"blanc",
    u"jyta":"bleu?",
    "syed":"sage[SG]",
    "syedi":"sage-i[PL]",
    "syedh":"sage-h",
    "syedix":"sage-DEF",
    "yoyyi":"petit",
    "wcta":"petit?",
    "uxziqyod":"merveilleux",
    "uxziqyodi":"merveilleux-i",
    "uxziqyodh":"merveilleux-h",
    "rozuxziqyod":"merveilleux",
    "rozuxziqyodi":"merveilleux-i",
    "ykssiyod":"ensemble",
    "ykssiyodh":"ensemble",
    "ykssiyodi":"ensemble",
    "uhasxicyodi":"ingrat",
    "ixiwhi":u"seul/unique?",    
    "whoyyi":u"calme/tranquille?",    
    "whoyyizi":u"calme/tranquille?",    
    "whoyyiw":u"calme/tranquille?",    
    "whoyyih":u"calme/tranquille?",   
    "hqe":u"loyal",
    "uhqe":u"dé-loyal",
    "wacci":u"bien?",
    "waccix":u"bien?",
    "ayhwacci":u"pas-bien?",
    "ayyiwaccix":u"?pas-bien?",
    "dezi":u"bon[??]",
    "wheq":u"grand[??]",
    "cteqs":u"sombre[??]",
    "dacyi":u"vieux[??]",
    "dacyiw":u"vieux[??]",
    "wtezi":u"doux[??]",
    "exzi":u"mauvais[??]",
    "pyoxs":u"intelligent/avisé[SG]",
    "pyoxsi":u"intelligent/avisé[PL?]",
    "wvuqdhi":u"demander/répondre?-hi[PST]",
    "wvowi":"dormir",
    "hewwih":"dire???-hi[PST]",
    "sayzh":"s'appeler?-h",
    "rawsi":"se laver[FUT]",
    "yirizi":"vivre-zi[PST.3PL?]",
    "jyirih":"vivre-ih[PST.3PL?]",
    "zqossi":"boire[FUT]",
    u"ñreq":u"PRO.3SG-être[PRS]",
    u"ñric":u"PRO.3SG-être[PST?]",
    u"ñixzi":u"être[PST?]",
    u"raq":u"être-COP[PST]",
    u"ñraz":u"PRO.3SG-être/avoir[?]",
    u"ñarzi":"PRO.3SG-avoir[PST?]",
    u"wadzi":"dire[PST?]",
    u"ztezi":"mourir[PST]",
    u"ztezix":"mort-DEF",
    u"ztezixw":"mort-DEF-GEN",
    u"ñax":"PRO.3SG",
    "zi":"PRO.3PL?/DEF.PL",
    "ziq":"PRO.3SG",
    u"ñaxw":"POSS.3SG",
    "dqihi":u"PRO?/Gretel?",        
    u"ñrow":"DONT/POSS?",
    u"zox":"POSS.2SG",
    u"zoh":"POSS.2SG",
    u"wox":"POSS.3SG",
    u"woxi":"POSS.3SG",
    "zix":"DEF[SG.x]",
    "zih":"DEF[SG.y]",
    "ed":"ET",
    "ix":"1-INDEF",
    "ih":"INDEF",
    "he":"2",
    "hqi":"3",
    "tiyzwhi":u"premier",
    "kxdwhi":u"second",
    "hqizoi":u"troisième",
    "wkr":"7",
    "wisw":"6?",
    "heyr":"12",
    "o":"DANS",
    "wec":"SUB",
    "cix":"MAIS",
    "ciz":"AVEC",
    "uzix":"SANS",
    "pqa":"DE(GEN)",
    "dos":"DE(PROV)",
    "wta":"COMME/AUSSI",
    }

In [295]:
for k,v in dictFoundGloss.iteritems():
    dictGlossWords[k]=k+"@"+v

In [296]:
texteHyphen=[]
texteGloss=[]
for nLineOrig,lineOrig in enumerate(texteBrut):
    lineHyphen=[]
    lineGloss=[]
    print nLineOrig+1
    for word in lineOrig.split(" "):
        if word:
            lineHyphen.append(dictHyphenWords[word])
            lineGloss.append(dictGlossWords[word])
    texteHyphen.append(" ".join(lineHyphen))
#    if nLineOrig+1==108:
#        print lineGloss
    texteGloss.append(" ".join(lineGloss))
    print texteHyphen[-1]
    print
    if texteHyphen[-1]!=texteGloss[-1]:
        print texteGloss[-1]
    print

1
001 pqte-se-xdix iyyiq tiqxñixqos

001 pqtesexdix@grenouille+roi-DEF iyyiq tiqxñixqos

2
o dacyi zadi zixdaxd cax suxxi pta woxi texwsiq evpkyzh yirizi ziq ix sexdi wec ñarzi ix zahhiq ziq raq wta zityod ah wiyr weyix ziq zed ñaq wih wta cidih stexh uxzqi-zi-w ñriq daxd zix wsoxxizi vta ñixzi

o@DANS dacyi@vieux[??] zadi zixdaxd@temps[DEF.PL] cax suxxi pta woxi@POSS.3SG texwsiq evpkyzh yirizi@vivre-zi[PST.3PL?] ziq@PRO.3SG ix@1-INDEF sexdi@roi wec@SUB ñarzi@PRO.3SG-avoir[PST?] ix@1-INDEF zahhiq@fille ziq@PRO.3SG raq@être-COP[PST] wta@COMME/AUSSI zityod ah wiyr weyix ziq@PRO.3SG zed ñaq wih wta@COMME/AUSSI cidih stexh uxzqi-zi-w ñriq daxd@temps zix@DEF[SG.x] wsoxxizi vta ñixzi@être[PST?]

3
o xtiqñizix ap wyehhih yta ziq ix wheq cteqs wser ed ziq oxzi raq ziq ix jqtexz uxziq ix dacciy yoxz

o@DANS xtiqñizix ap wyehhih yta ziq@PRO.3SG ix@1-INDEF wheq@grand[??] cteqs@sombre[??] wser@forêt ed@ET ziq@PRO.3SG oxzi raq@être-COP[PST] ziq@PRO.3SG ix@1-INDEF jqtexz uxziq ix@1-INDEF dacciy yoxz

365
qodhod dezh wraqizi ñax xu iq zix ixi xah peqjo

qodhod dezh wraqizi ñax@PRO.3SG xu iq zix@DEF[SG.x] ixi xah peqjo

366
zi he axzqi dtaq edwta xes

zi@PRO.3PL?/DEF.PL he@2 axzqi dtaq edwta xes

367
ñax dos wta uz hoy rtiqhix ziq dteqzi wheqi tetxi

ñax@PRO.3SG dos@DE(PROV) wta@COMME/AUSSI uz hoy rtiqhix ziq@PRO.3SG dteqzi wheqi tetxi

368
tid hqeizi qodhodxes ayzqod tid wsuyyi wi zod odix wadzi ñax ñaq zu xu ytiqh ñraz zih iq ah rtiqi jaxdi

tid hqeizi qodhodxes ayzqod tid wsuyyi wi zod odix wadzi@dire[PST?] ñax@PRO.3SG ñaq zu xu ytiqh ñraz@PRO.3SG-être/avoir[?] zih@DEF[SG.y] iq ah rtiqi jaxdi

369
xit wraqizi zqixdix zih ñttiyviq wyih ossi

xit wraqizi zqixdix zih@DEF[SG.y] ñttiyviq wyih ossi

370
jaqi ziq zed raq ix ziq royyi ytiqi cod zih

jaqi ziq@PRO.3SG zed raq@être-COP[PST] ix@1-INDEF ziq@PRO.3SG royyi ytiqi cod zih@DEF[SG.y]

371
zix xtiwhi xah dos ñax odix oxz o zih dacyi wyeh wahhi wod ñix hoy oyzix ed jidkxzhi wox dacyi rowi jaqi tid xu suxxi jyori jaxdi

zix@DEF[SG.x] x

zi@PRO.3PL?/DEF.PL axzqi htixiqi sxuqqi-zi ed@ET wadzi@dire[PST?] xu jqtixz-iq ñax@PRO.3SG erixostejih sexdixw jqkyyuvw-steqhi

617
cix sexdix wadzi cax sax ossi rozi ñraz zih iq dezh peq

cix@MAIS sexdix@roi-DEF wadzi@dire[PST?] cax sax ossi rozi ñraz@PRO.3SG-être/avoir[?] zih@DEF[SG.y] iq dezh peq

618
yaz ñac dta o pqiz ñax iq cox hqe teñaxxiw

yaz ñac dta o@DANS pqiz ñax@PRO.3SG iq cox hqe@loyal teñaxxiw

619
xu jyir jqkyyuvvih pitqih ed zaxwi-x jidkxzhi

xu jyir jqkyyuvvih pitqih ed@ET zaxwi-x jidkxzhi

620
jquzix zaxwizi edwta cix vta ix daxd jyir ñux yodjyid ed payzh ec wec ztez

jquzix zaxwizi edwta cix@MAIS vta ix@1-INDEF daxd@temps jyir ñux yodjyid ed@ET payzh ec wec@SUB ztez

621
za wvqaxd zix hqe teñaxxiw ñuqhod pqic ytephizi ñixzi ev ed jaq ñixzi oxz o ih rtiqiywi

za wvqaxd zix@DEF[SG.x] hqe@loyal teñaxxiw ñuqhod pqic ytephizi ñixzi@être[PST?] ev ed@ET jaq@enfant ñixzi@être[PST?] oxz o@DANS ih@INDEF rtiqiywi

622
ziq yadzi ñax wod vta sxti wudi-zi hqi jyezzqtajiq ap ñixzi


868
riz wozix ap ñuwih raq ziq ix yoyyi ñari ñreqo ziq whez heyr yoyti-q ed za vodix diqxi royyi dteqi woxi jqtezqi ix dytizi dos ñux uz peq ah vyussi zi heyr jyecwhiq peq ah dori zic ñriq ix xtaq zi ñarzi wvowh

riz wozix ap ñuwih raq@être-COP[PST] ziq@PRO.3SG ix@1-INDEF yoyyi@petit ñari ñreqo ziq@PRO.3SG whez heyr@12 yoyti-q ed@ET za vodix@jeune.fille-DEF diqxi royyi dteqi woxi@POSS.3SG jqtezqi@frère ix@1-INDEF dytizi dos@DE(PROV) ñux uz peq ah vyussi zi@PRO.3PL?/DEF.PL heyr@12 jyecwhiq peq ah dori zic ñriq ix@1-INDEF xtaq zi@PRO.3PL?/DEF.PL ñarzi@PRO.3SG-avoir[PST?] wvowh

869
cix o wacci xu ñux vyussizi zic jyir zi heyr jqtezqi peqraxzyizi hoy qarxi ed pytet wsqodixzi ñixeriq wserix

cix@MAIS o@DANS wacci@bien? xu ñux vyussizi zic jyir zi@PRO.3PL?/DEF.PL heyr@12 jqtezqi@frère peqraxzyizi hoy qarxi@corbeau-PL? ed@ET pytet wsqodixzi ñixeriq wserix@forêt-DEF

870
ñuwih ed ñarix raq edwta peqwruxzih

ñuwih ed@ET ñarix raq@être-COP[PST] edwta peqwruxzih

871
xu raq zix whassiyw vodi ñi

1119
013 zi hqi wcta ctixz o wserix

013 zi@PRO.3PL?/DEF.PL hqi@3 wcta@petit? ctixz o@DANS wserix@forêt-DEF

1120
ziq raq ixdaxd ix caxz ñrow sexi ztezi ed ix sexi ñrow caxz ztezi

ziq@PRO.3SG raq@être-COP[PST] ixdaxd@temps[INDEF?] ix@1-INDEF caxz@homme ñrow@DONT/POSS? sexi@femme ztezi@mourir[PST] ed@ET ix@1-INDEF sexi@femme ñrow@DONT/POSS? caxz@homme ztezi@mourir[PST]

1121
caxzix ñarzi ix zahhiq ed sexix ñarzi edwta ix zahhiq

caxzix@homme-DEF ñarzi@PRO.3SG-avoir[PST?] ix@1-INDEF zahhiq@fille ed@ET sexix@femme-DEF ñarzi@PRO.3SG-avoir[PST?] edwta ix@1-INDEF zahhiq@fille

1122
vodijteqxixi sixzhi ñoxaxzix ed ix zad za zi ñarzi rtiqih uzi ah wvazwiqi waccix dos zi ñtic hoy sexix

vodijteqxixi sixzhi ñoxaxzix ed@ET ix@1-INDEF zad@jour za zi@PRO.3PL?/DEF.PL ñarzi@PRO.3SG-avoir[PST?] rtiqih uzi ah wvazwiqi waccix@bien? dos@DE(PROV) zi@PRO.3PL?/DEF.PL ñtic hoy sexix@femme-DEF

1123
ñux wadzi wta hoy caxzixw zahhiq wod hoy zox paq ah ñrow ñax dophiq wod ciz cod wta wsay zu ñriq ceqdix rawsi 

...

sexdix@roi-DEF pos ykwh hoy edwta ah vqteri wox@POSS.3SG ykssi ed@ET jidar wod vta rit

11361
za ñax sec hoy pyezix hed caxzix ñac vta qkddix cix ñax raq xihev zix pteqwhi ziq sec iphiq ah ñaxw ñarzi rtiqih ziq ed cozh uzi o raxzih wahhi caxzix ñac xiz ed ytej wox rit

za ñax@PRO.3SG sec hoy pyezix hed caxzix@homme-DEF ñac vta qkddix cix@MAIS ñax@PRO.3SG raq@être-COP[PST] xihev zix@DEF[SG.x] pteqwhi ziq@PRO.3SG sec iphiq ah ñaxw@POSS.3SG ñarzi@PRO.3SG-avoir[PST?] rtiqih ziq@PRO.3SG ed@ET cozh uzi o@DANS raxzih wahhi caxzix@homme-DEF ñac xiz ed@ET ytej wox@POSS.3SG rit

11362
sexdix zqusxizi ed ñaxw dophi-zi wod ciz sexdizahhiqix ed jyir sexdi eriq ñiyi qodih

sexdix@roi-DEF zqusxizi ed@ET ñaxw@POSS.3SG dophi-zi wod ciz@AVEC sexdizahhiqix@roi+château-DEF ed@ET jyir sexdi@roi eriq ñiyi qodih

11363
166 zix whtiqsi ñaxw

166 zix@DEF[SG.x] whtiqsi ñaxw@POSS.3SG

11364
ziq raq ixdaxd ix caxz ed ix sexi wec ñarzi ix ixiwhi wtex

ziq@PRO.3SG raq@être-COP[PST] ixdaxd@temps[INDEF?] ix@1-INDEF c


wixdix raq@être-COP[PST] ap iypixjix ed@ET vuqvuqqtez pytety-w-ñ-occiy ed@ET vta ix@1-INDEF whey riz wozix ap whez zi@PRO.3PL?/DEF.PL ytis-qiwhi wcta@petit? duyzj-qeziqizi htepyiq

11615
vodix hqeizi zih raq ix zqtec cix za hqtazhi hqi qodh-sytizhi htixiqi oxz ed wvuqdhi ñraz ñux ñarzi ah jipayi

vodix@jeune.fille-DEF hqeizi zih@DEF[SG.y] raq@être-COP[PST] ix@1-INDEF zqtec cix@MAIS za hqtazhi hqi@3 qodh-sytizhi htixiqi oxz ed@ET wvuqdhi@demander/répondre?-hi[PST] ñraz@PRO.3SG-être/avoir[?] ñux ñarzi@PRO.3SG-avoir[PST?] ah jipayi

11616
o sax dezh dta wadzi vodix xu whtaq tid ev ed sediq wuvvi hoy zix dacyi caxz ed pezqi-q ñtexix ñaxix ed zix jyassizi se

o@DANS sax dezh dta wadzi@dire[PST?] vodix@jeune.fille-DEF xu whtaq tid ev ed@ET sediq wuvvi hoy zix@DEF[SG.x] dacyi@vieux[??] caxz@homme ed@ET pezqi-q ñtexix ñaxix ed@ET zix@DEF[SG.x] jyassizi se

11617
ñux rixzhi wod ec peq ah wi ec zix dacyi raq whtaih ev cix za wta ñux ah ziq yta ix uxd wcus caxz o ñaxw wixd

ñux rixzhi wod ec peq

11855
za iq ñaxw peqw-haxz wrad ed ñax jtiqiq wod htajiyod-h az wta jteqxixi yiq az ñac

za iq ñaxw@POSS.3SG peqw-haxz wrad ed@ET ñax@PRO.3SG jtiqiq wod htajiyod-h az wta@COMME/AUSSI jteqxixi yiq az ñac

11856
177 ztezi-xw wixzijuz

177 ztezixw@mort-DEF-GEN wixzijuz

11857
peq caxdi caxdi taq wozix raxzq-izi ix sticvi ñix az yaxz-iritix

peq caxdi caxdi taq wozix raxzq-izi ix@1-INDEF sticvi ñix az yaxz-iritix

11858
vyuzwiyod wvqaxd ix pqicciz caxz ñixocez ñac ed qtajhi ñeyzh ossi ih wsqozh roziqi

vyuzwiyod wvqaxd ix@1-INDEF pqicciz caxz@homme ñixocez ñac ed@ET qtajhi ñeyzh ossi ih@INDEF wsqozh roziqi

11859
ñraz peq xedih wadzi sticvix zu yoyyi pkq wec tid sax sxuwi ciyyic poxdqixi reriq ah hqtizi o ritix peq cod

ñraz@PRO.3SG-être/avoir[?] peq xedih wadzi@dire[PST?] sticvix zu yoyyi@petit pkq wec@SUB tid sax sxuwi ciyyic poxdqixi reriq ah hqtizi o@DANS ritix peq cod

11860
ñric iq zu wozix zu hteq hayi wta zqowhodh

ñric@PRO.3SG-être[PST?] iq zu wozix zu hteq hayi wta@COMME/AUSSI zq

12095
cix sexdix royyi ñari ah ñux wsuyyi wodi xedih

cix@MAIS sexdix@roi-DEF royyi ñari ah ñux wsuyyi wodi xedih

12096
hoy wozwh wadzi ñux zix zityodwhi caz wcadiq cod ossi uzix wayh

hoy wozwh wadzi@dire[PST?] ñux zix@DEF[SG.x] zityodwhi caz wcadiq cod ossi uzix@SANS wayh

12097
tid ñeyziq wta cidih ap zod wec ap wayh

tid ñeyziq wta@COMME/AUSSI cidih ap zod wec@SUB ap wayh

12098
sexdix jyir qawixzi ed wadzi xtaq zu ñeyziq wta cidih ap cod wec ap wayh wsay zox stiqyodñiz edwta jyori jiyt-exx-ih ciz wayh

sexdix@roi-DEF jyir qawixzi ed@ET wadzi@dire[PST?] xtaq zu ñeyziq wta@COMME/AUSSI cidih ap cod wec@SUB ap wayh wsay zox@POSS.2SG stiqyodñiz edwta jyori jiyt-exx-ih ciz@AVEC wayh

12099
ñax ziyhi xu qodih ciyyic zi he tiyzwhi

ñax@PRO.3SG ziyhi xu qodih ciyyic zi@PRO.3PL?/DEF.PL he@2 tiyzwhi@premier

12100
ziqvta jyir ziq juxzih ix wtis ciz wayh vta qkddix ap reqiw kxdwhi ed he saqyi ctahhi pteqi ñixzi uz o zix wheqi royzi wser

ziqvta jyir ziq@PRO.3SG juxzih ix@1-INDEF wtis ciz@AVE

za peqtaqih ixdaxd@temps[INDEF?] odix ñarzi@PRO.3SG-avoir[PST?] ñeyzh woh oxzhed zqed zi@PRO.3PL?/DEF.PL jiddi ix@1-INDEF zad@jour ap whiz ciz@AVEC ziqiw ñteqziq ed@ET hoyptiyzih royyi ah zi@PRO.3PL?/DEF.PL sec o@DANS xtiqñizix ap ñoxaxzix

12326
ttidiqix wta o zih ptiqxi ix ñteq-z vta ix jtiqd-wsqtixh ed zqir woxi ptaq o zix qihxoxd

ttidiqix wta@COMME/AUSSI o@DANS zih@DEF[SG.y] ptiqxi ix@1-INDEF ñteq-z vta ix@1-INDEF jtiqd-wsqtixh ed@ET zqir woxi@POSS.3SG ptaq o@DANS zix@DEF[SG.x] qihxoxd

12327
zi ctez-hiw xu o ix zay zi sixzhi ossi ñoxaxzix cix raq zed dyazi eriq ossi ciqi ah rtiqi wta ayixi

zi@PRO.3PL?/DEF.PL ctez-hiw xu o@DANS ix@1-INDEF zay zi@PRO.3PL?/DEF.PL sixzhi ossi ñoxaxzix cix@MAIS raq@être-COP[PST] zed dyazi eriq ossi ciqi ah rtiqi wta@COMME/AUSSI ayixi

12328
zi puydh-iw xu ñriq zad uz ciz ziqiw ñteq-z

zi@PRO.3PL?/DEF.PL puydh-iw xu ñriq zad@jour uz ciz@AVEC ziqiw ñteq-z

12329
qih cidih hayhi zi ossi waccix cix zi pteyhi wod zed yozh dyaziqi

qih cidih hayhi zi@PRO.3

## Concordances

### Texte brut

In [79]:
def displayConcordance(concordance,fenetre=50):
    blancGauche=concordance.startswith(" ")
    blancDroite=concordance.endswith(" ")

    dfConcordance=pd.DataFrame(columns=["conte","num","gauche",concordance,"droite"])
    p=re.compile("(.*)(%s)(.*)"%concordance,re.U)
    for nLine,line in enumerate(texteBrut[:]):
        occurrences=re.findall(p,line)
        if occurrences:
            for occurrence in occurrences:
                if len(occurrence[0])>fenetre:
                    gauche=occurrence[0][-fenetre:]
                else:
                    gauche=occurrence[0]
                if not blancGauche and gauche and gauche[-1]!=" ":
                    gauche=gauche+"-"

                if len(occurrence[2])>fenetre:
                    droite=occurrence[2][:fenetre]
                else:
                    droite=occurrence[2]
                if not blancDroite and droite and droite[0]!=" ":
                    droite="-"+droite

                dfLine={"conte":lineConte[nLine],"num":nLine+1,"gauche":gauche,concordance:occurrence[1],"droite":droite}
                dfConcordance=dfConcordance.append(dfLine,ignore_index=True)

    # dfStyler=dfConcordance.style.set_properties(subset=[concordance],**{'font-weight': 'bold'})
    dfStyler=dfConcordance.style.set_properties(subset=['gauche'],**{'text-align': 'right'})
    dfStyler=dfConcordance.style.set_properties(subset=['droite'],**{'text-align': 'left'}).hide_index()
    display(dfStyler)
    return


In [38]:
displayConcordance(" whoyyi")

conte,num,gauche,whoyyi,droite
001,69,hizi xu sexdiwtexxix ed vqoxwiwwix oxz o redxix ed,whoyyi,-zi wod jadriz zic ed ñaxw ñtiqhi wruycizi ap dytiz
002,84,shi wod ytixdi ec ed ixziyod wadzi zix ro sax ossi,whoyyi,zix xedih wosqiqi whiz ixz o soqsix ziq iq ziq ox
003,163,ñux har wta,whoyyi,cix xkwdiqqodñizix yez ñixzi ossi ñari pqiz
003,216,rzi tizh zih ed coxowhqixi peqyaxdhi ah ñux wsuyyi,whoyyi,-w peq ix zecwhey
003,225,ñux jyir,whoyyi,-h peq ix zecwhey ed za ñux ossi suxxi peqwraqi wod
004,234,x kxdwhi raq wta zuc ah ziq wyih ossi raq xedih ah,whoyyi,ev ciz ñac
004,248,c ed wadzi ah ñax wyih ossi rozwhi ñraz ñax wsuyyi,whoyyi,ev ciz wox kxdwhi wtex ñax raq wta zuc wta zih os
004,263,xshi zidxix ed dar ossi ix ykz pqa wod ed whez wta,whoyyi,wec ec ñax raq ap whix
004,324,ho zed,whoyyi,wadzi ñaxw sexi ziq iq ayyiqizi wta caxdi ziq ñaq
004,337,hixix dos zqixdix ziqoxz htixzhi wod ih syaqh jtay,whoyyi,-zi ñteryijtixsix ciz sxorix riz wozix ap ed wahhi


In [39]:
displayConcordance(" wvtiqqi")

conte,num,gauche,wvtiqqi,droite
011,988,x za qtazkqih wadzi tid zteq ap weqd ñrow tid wsay,wvtiqqi,-w oxzi ñiq suxxi ñux ossi axzih ixz ciz huxdh ñtiq
019,1716,ñax hqeizi ñax ñarzi ñteqh dayh ed,wvtiqqi,-zi tetxixi rozh ev ed wadzi ñraz iq zih zed zu wod
057,4693,ax ptiqxi zih jtiqd ziq yoddiq uzixpeq coh wyeh ed,wvtiqqi,-q uzwodhix
060,5163,ap ritix peq zix cix za zix sec hoy wsoyzradhiqxi,wvtiqqi,-zi zi ritix peq zix ciz ziqiw jtewwiq ed royyi oss
060,5173,wsoyzradhiqxi royyi,wvtiqqi,ritix peq zix cix zix uzwhtezhi jyeh ih jqtey wta
071,5913,hui ñreq duyrih ed zteqix raq ap tiqx ed roxzuiqxi,wvtiqqi,-h ciz tiqxwhtixdiq
166,11395,zi dos xu uz ap ñuyix ed zih sax xes rtiqi ah ñaxw,wvtiqqi,-zi tetxixi ev za ñax pqa cteqsih sec uz o zih syaq
166,11459,i ed za ñux ossi royyi dophi wod ciz ñac ñarzi ñax,wvtiqqi,-h ñixzi oxzi o syovvix ed wah zix exzwsajwpuyzi zr
169,11625,zi iq,wvtiqqi,-h oxzi o stiyziqix wraqizi ñax o ceqdix wsay zi pt
172,11710,wh azwsoyhi zic ziq diqxi royyi rtiqi waccix iyyiq,wvtiqqi,-zi ritix peq zic


In [162]:
conte="031"
if conte in tradContes:
    print "\n".join(conteLines[conte])

9471 031 la jeune fille sans mains
9472 il était une fois il y a quelques jours à l époque où la farine des villageois était écrasée à la meule de pierre un meunier qui avait connu des temps difficiles
9473 il ne lui restait plus que cette grosse meule de pierre dans une remise et derrière un superbe pommier en fleur
9474 un jour tandis qu il allait dans la forêt couper du bois mort avec sa hache au tranchant d argent un curieux vieillard surgit de derrière un arbre
9475 a quoi bon te fatiguer à fendre du bois
9476 dit il
9477 ecoute si tu me donnes ce qu il y a derrière ton moulin je te ferai riche
9478 qu y a t il derrière mon moulin sinon mon pommier en fleurs
9479 pensa le meunier
9480 il accepta donc le marché du vieil homme
9481 dans trois ans je viendrai chercher mon bien gloussa l étranger avant de disparaître en boitant derrière les arbres
9482 sur le sentier en revenant le meunier vit son épouse qui volait à sa rencontre les cheveux défaits le tablier en bataille
9483 mon épo

### Texte glosé

In [41]:
def displayGlossConcordance(concordance,fenetre=50):
    blancGauche=concordance.startswith(" ")
    blancDroite=concordance.endswith(" ")

    dfConcordance=pd.DataFrame(columns=["conte","num","gauche",concordance,"droite"])
    p=re.compile("(.*)(%s)(.*)"%concordance,re.U)
    for nLine,line in enumerate(texteGloss[:]):
        occurrences=re.findall(p,line)
        if occurrences:
            for occurrence in occurrences:
                if len(occurrence[0])>fenetre:
                    gauche=occurrence[0][-fenetre:]
                else:
                    gauche=occurrence[0]
                if not blancGauche and gauche and gauche[-1]!=" ":
                    gauche=gauche+"-"

                if len(occurrence[2])>fenetre:
                    droite=occurrence[2][:fenetre]
                else:
                    droite=occurrence[2]
                if not blancDroite and droite and droite[0]!=" ":
                    droite="-"+droite

                dfLine={"conte":lineConte[nLine],"num":nLine+1,"gauche":gauche,concordance:occurrence[1],"droite":droite}
                dfConcordance=dfConcordance.append(dfLine,ignore_index=True)

    # dfStyler=dfConcordance.style.set_properties(subset=[concordance],**{'font-weight': 'bold'})
    dfStyler=dfConcordance.style.set_properties(subset=['gauche'],**{'text-align': 'right'})
    dfStyler=dfConcordance.style.set_properties(subset=['droite'],**{'text-align': 'left'}).hide_index()
    display(dfStyler)
    return

In [297]:
displayGlossConcordance(u" cuxz",50)

conte,num,gauche,cuxz,droite
001,21,ozh iphiq ev odix ciz@AVEC sudyix@balle-DEF o@DANS,cuxz,-ix
001,54,x@grenouille-DEF yez wod cazix wcadi cix@MAIS ñriq,cuxz,-puyz jyir wozzixzi ñixzi@être[PST?] o@DANS ñaywix
002,94,USSI wod ec ed@ET weyizi wod ed@ET wyossizi wod ec,cuxz,-ix ñriq daxd@temps zix@DEF[SG.x] htixshi vta pizhi
002,137,roy zu ñeyzi,cuxz,qtajhi sahhix@chat-DEF ñrow@DONT/POSS? zu wodiq i
004,308,ziqevvi ñaq ossi ixdaxd@temps[INDEF?] dozih yussi,cuxz,-ix ev cix@MAIS ñaq rtiqih zucci xes hoy ah yazi zi
013,1160,être[PST?] ah ziq@PRO.3SG payziq duyzwhkssiq uz ap,cuxz,-ix vta ñixzi@être[PST?] peq ñriqh eqz ñux wodiq wa
013,1164,T?] dezaphix payzh ziq@PRO.3SG iq duyzwhkssi uz ap,cuxz,-ix vta ñixzi@être[PST?]
013,1165,dzi@dire[PST?] payzh duyzw-h-kssi-qxi uz ap ñixziw,cuxz,
013,1177,ziq@PRO.3SG wvqoxdiq ix@1-INDEF wsqu-vhuzwi uz ap,cuxz,-ix vta ñixzi@être[PST?] peq ñriqh eqz ñux wodiq wa
013,1180,xd ziq@PRO.3SG ix@1-INDEF wsqu-vhuzwi uz ap ñixziw,cuxz,peq ñriqh eqz ñux wadzi@dire[PST?] ed@ET ayyi cix


In [260]:
displayGlossConcordance(u"^\d\d\d ",80)

conte,num,gauche,^\d\d\d,droite
001,1,,001,pqtesexdix@grenouille+roi-DEF iyyiq tiqxñixqos
002,80,,002,sah@chat ed@ET cuw@souris o@DANS jeptiyyiwsaj
003,140,,003,caqoijaqxih
004,232,,004,irixhkq-ih ec ix@1-INDEF ziq@PRO.3SG zqed uz peq ah ytiqi pqkdh ah sixzi
005,445,,005,uyrix@loup-DEF ed@ET zi@PRO.3PL?/DEF.PL wkr@7 dizisoz@chevreau-PL
006,505,,006,zix@DEF[SG.x] hqe@loyal teñaxxiw
007,654,,007,zix@DEF[SG.x] dezi@bon[??] ñaxziy
008,744,,008,zix@DEF[SG.x] uxziqyodi@merveilleux-i wvoyyicaxz@violon
009,795,,009,zi@PRO.3PL?/DEF.PL heyr@12 jqtezqi@frère
010,900,,010,vas


## Calculer une hiérarchie de signatures morphologiques
Les signatures morphologiques de Linguistica correspondent à un paradigme flexionnel réduit aux variantes attestées dans le corpus pour un même lexème. Hiérarchiser les signatures permet d'étudier les regroupements possibles entre paradigmes partiels.

In [15]:
morphGraph=nx.Graph()
sigGraph=nx.DiGraph()

In [16]:
for sig in lxaCorpus.signatures():
    for a,b in combinations(sig,2):
        if (a,b) not in morphGraph.edges:
            morphGraph.add_edge(a,b,weight=1)
        else:
            morphGraph[a][b]["weight"]+=1

### Cliques maximales parmi les signatures

In [17]:
maxCliques=list(nx.find_cliques(morphGraph))
# maxCliques

In [18]:
dCliques={tuple(c):set() for c in maxCliques}
for sig in lxaCorpus.signatures():
    sSig=set(sig)
    for clique in dCliques:
        sClique=set(clique)
        if sSig.intersection(sClique)==sSig:
            dCliques[clique].add(sig)
print "Les cliques maximales et leurs sous-cliques sont :"
print
for cMax,lCliques in dCliques.iteritems():
    if len(lCliques)>1:
        print cMax,":"
        print "\t",list(lCliques)
    else:
        print cMax
    print

Les cliques maximales et leurs sous-cliques sont :

(u'NULL', u'i', u'iw') :
	[(u'NULL', u'i'), (u'NULL', u'i', u'iw')]

(u'NULL', u'i', u'\xf1iz') :
	[(u'NULL', u'i'), (u'NULL', u'i', u'\xf1iz')]

(u'NULL', u'ci')

(u'NULL', u'i', u'ix', u'ixi', u'ixw') :
	[(u'NULL', u'i', u'ix', u'ixi'), (u'NULL', u'ix'), (u'NULL', u'i', u'ix'), (u'NULL', u'ix', u'ixw'), (u'NULL', u'i'), (u'NULL', u'i', u'ix', u'ixi', u'ixw'), (u'NULL', u'ixi')]

(u'hi', u'i', u'h') :
	[(u'h', u'i'), (u'h', u'hi', u'i')]

(u'NULL', u'q', u'ywi') :
	[(u'NULL', u'q'), (u'NULL', u'q', u'ywi')]

(u'NULL', u'qi', u'whi') :
	[(u'NULL', u'qi'), (u'NULL', u'qi', u'whi'), (u'NULL', u'whi')]

(u'NULL', u'zi', u'q', u'h', u'xzi', u'w') :
	[(u'xzi', u'zi'), (u'NULL', u'h', u'w'), (u'q', u'zi'), (u'NULL', u'q', u'w'), (u'NULL', u'h', u'q'), (u'q', u'xzi', u'zi'), (u'NULL', u'q', u'zi'), (u'NULL', u'q'), (u'NULL', u'h', u'zi'), (u'h', u'q'), (u'h', u'q', u'zi'), (u'NULL', u'q', u'w', u'xzi'), (u'NULL', u'w'), (u'NULL', u'zi'), (u'

In [ ]:
sigMax={}
for k,l in dCliques.iteritems():
    for e in l:
        if e not in sigMax:
            sigMax[e]=set()
        sigMax[e].add(k)
sigMax[(u'NULL', u'ix')]

In [24]:
lxaCorpus.stems()
lxaCorpus.affixes()
dSignatureStems=lxaCorpus.signatures_to_stems()

In [23]:
keySignatures=[]
for k in lxaCorpus.signatures_to_stems().keys():
    if len(k)>4:
        keySignatures.append(k)

In [29]:
for k in keySignatures:
    print ", ".join(k)
    print "\t",", ".join(lxaCorpus.signatures_to_stems()[k])
    print

NULL, h, q, w, zi
	whoyyi, pitqi, wvtiqqi, ñixhi, tadi, ñuddi

NULL, si, sih, siq, sizi
	ctiq, ztis, vyus, rtis, hqks

NULL, h, q, xzi, zi
	yowhi, whkqhi, vawwi, wrtecci, ñevvi, qkwhi, jaxsi, sqaryi

NULL, i, ih, iq, izi
	rixh, yteph, wraq, redh, texws, ñitw

NULL, i, ix, ixi, ixw
	wrixz, ñisw, zqixd, jtteqx, ñiwh

NULL, h, q, x, zi
	yazi, paxdi, vqteri, zaxwi, reswi, ñassi, syovvi, ñroyi, qidxi, rtadi



In [ ]:
lxaCorpus.signatures_to_words()
lxaCorpus.affixes_to_signatures()

In [ ]:
lxaCorpus.stems_to_signatures()["zesheq"]

In [ ]:
lxaCorpus.words_in_signatures()
lxaCorpus.words_to_signatures()
lxaCorpus.words_to_sigtransforms()

## Words manifolds and syntactic word neighborhood

In [ ]:
# sorted(lxaCorpus.words_to_neighbors().keys())
lxaCorpus.words_to_neighbors()["jexzix"]

In [ ]:
network=lxaCorpus.neighbor_graph()
# plt.figure(1,figsize=(100,100))
# nx.draw(network)
# plt.show()

In [ ]:
lxaCorpus.words_to_contexts()

## Phonology

In [ ]:
lxaCorpus.phone_unigram_counter()
lxaCorpus.phone_bigram_counter()
lxaCorpus.phone_trigram_counter()

## Voisinages

In [22]:
print (", ".join(lxaCorpus.words_to_neighbors()["wvuqdhi"]))

htixshi, qtajhi, jyir, ctahhi, hed, roy, ñaq, royyi, ñteqhi
